# Parallelism in Computer Vision

## Setup

### Imports

In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from pathlib import Path
import zipfile
import urllib.request
from typing import List

### Setup Data paths for Caltech101 dataset

In [2]:
DATA_ROOT = Path("../data")
ARCHIVE_URL = "https://www.kaggle.com/api/v1/datasets/download/imbikramsaha/caltech-101"
ARCHIVE_PATH = DATA_ROOT / "caltech101.zip"
EXTRACT_DIR = DATA_ROOT / "caltech-101"


In [3]:
def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f}s")
        return result
    return wrapper

### Download Caltech101 dataset

In [ ]:
def ensure_caltech101():
    if EXTRACT_DIR.exists() and any(EXTRACT_DIR.iterdir()):
        print("Caltech101 dataset already exists.")
        return
    print("Downloading Caltech101 archive...")
    urllib.request.urlretrieve(ARCHIVE_URL, ARCHIVE_PATH)
    print("Extracting Caltech101 archive...")
    with zipfile.ZipFile(ARCHIVE_PATH, "r") as zip_ref:
        zip_ref.extractall(path=DATA_ROOT)
    print("Caltech101 dataset extracted to", EXTRACT_DIR)

ensure_caltech101()


Extracting Caltech101 archive...
Caltech101 dataset extracted to ../data/caltech101


### Sample Images

In [5]:
def sample_images(root: Path, limit: int=500) -> List[Path]:
    extensions = {".jpg", ".jpeg", ".png", ".bmp"}
    paths = [path for path in root.rglob("*") if path.suffix.lower() in extensions]
    return paths[:limit]

paths = sample_images(EXTRACT_DIR)
len(paths), paths[:5]


(500,
 [PosixPath('../data/caltech-101/yin_yang/image_0042.jpg'),
  PosixPath('../data/caltech-101/yin_yang/image_0002.jpg'),
  PosixPath('../data/caltech-101/yin_yang/image_0053.jpg'),
  PosixPath('../data/caltech-101/yin_yang/image_0009.jpg'),
  PosixPath('../data/caltech-101/yin_yang/image_0019.jpg')])

## Serial Preprocessing

Resize, grayscale, Sobel edges.

In [6]:
def load_resize(path, size=(224,224)):
    """
    Open an image, convert it to RGB to ensure we have 3 channels, and resize it.
    Returns a NumPy array of shape (height, width, 3).
    """
    with Image.open(path) as img:
        img = img.convert("RGB").resize(size, Image.BILINEAR)
        return np.array(img, dtype=np.uint8)

def to_gray(img_rgb):
    """
    Convert an RGB image to grayscale using perceptual weights.
    Returns a NumPy array of shape (height, width).
    """
    # Use ellipsis (...) to select all pixels
    r, g, b = img_rgb[..., 0], img_rgb[..., 1], img_rgb[..., 2]
    # Apply ITU-R 601 to calculate luminance
    # Cast float32 to avoid overflow later
    return (0.299 * r + 0.587 * g + 0.114 * b).astype(np.float32)

def sobel_edges(img_gray):
    """
    Apply the Sobel operator to a grayscale image of shape (height, width) to detect edges.
    Returns a NumPy array of the same shape as the input image.
    """
    # Create two 3x3 Sobel kernels
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]], dtype=np.float32)
    sobel_y = np.array([[ 1, 2, 1],
                        [ 0, 0, 0],
                        [-1,-2,-1]], dtype=np.float32)

    grad = np.pad(img_gray, ((1, 1), (1, 1)), mode='edge')
    grad_x = (
        grad[:-2, :-2] * sobel_x[0, 0] + grad[:-2, 1:-1] * sobel_x[0, 1] + grad[:-2, 2:] * sobel_x[0, 2] +
        grad[1:-1, :-2] * sobel_x[1, 0] + grad[1:-1, 1:-1] * sobel_x[1, 1] + grad[1:-1, 2:] * sobel_x[1, 2] +
        grad[2:, :-2] * sobel_x[2, 0] + grad[2:, 1:-1] * sobel_x[2, 1] + grad[2:, 2:] * sobel_x[2, 2]
    )
    grad_y = (
        grad[:-2, :-2] * sobel_y[0, 0] + grad[:-2, 1:-1] * sobel_y[0, 1] + grad[:-2, 2:] * sobel_y[0, 2] +
        grad[1:-1, :-2] * sobel_y[1, 0] + grad[1:-1, 1:-1] * sobel_y[1, 1] + grad[1:-1, 2:] * sobel_y[1, 2] +
        grad[2:, :-2] * sobel_y[2, 0] + grad[2:, 1:-1] * sobel_y[2, 1] + grad[2:, 2:] * sobel_y[2, 2]
    )
    return np.sqrt(grad_x**2 + grad_y**2).astype(np.float32)

In [7]:
def preprocess_one(path, size=(224, 224), bins=32):
    """
    Preprocess pipeline for one image.
    Returns a dictionary containing components for reuse.
    """
    img_resized = load_resize(path, size)
    img_gray = to_gray(img_resized)
    img_edges = sobel_edges(img_gray)

    hists = []
    for channel in range(3):
        hist_channel, _ = np.histogram(img_resized[..., channel], bins=bins, range=(0, 255))
        hists.append(hist_channel.astype(np.float32))

    color_hist = np.concatenate(hists, axis=0)

    return {
        "path": str(path),
        "resized": img_resized,
        "gray": img_gray,
        "edges": img_edges,
        "color_hist": color_hist,
        "size": size,
        "bins": bins
    }

def preprocess_all_serial(image_paths, size=(224, 224), bins=32, limit=None):
    """
    Preprocess pipeline for a list of images.
    Returns a list of dictionaries, each containing components for reuse.
    """
    if limit is not None:
        image_paths = image_paths[:limit]
    return [preprocess_one(path, size=size, bins=bins) for path in image_paths]


### Performance test on different resizing

Default size 224x224

In [8]:
t0 = time.perf_counter()
samples_big = preprocess_all_serial(paths, limit=None)
elapsed = time.perf_counter() - t0

print(f"Processed {len(samples_big)} samples in {elapsed:.2f} seconds.")

Processed 500 samples in 3.64 seconds.


112x112

In [9]:
t0 = time.perf_counter()
samples_medium = preprocess_all_serial(paths, limit=None, size=(112,112))
elapsed = time.perf_counter() - t0

print(f"Processed {len(samples_medium)} samples in {elapsed:.2f} seconds.")

Processed 500 samples in 1.36 seconds.


56x56

In [10]:
t0 = time.perf_counter()
samples_small = preprocess_all_serial(paths, limit=None, size=(56,56))
elapsed = time.perf_counter() - t0

print(f"Processed {len(samples_small)} samples in {elapsed:.2f} seconds.")

Processed 500 samples in 0.93 seconds.
